# 可視化

CNNにおいても可視化は重要である．このノートブックでは，学習と評価が終わったCNN再度読み込み，特徴ベクトルの可視化，フィルタの可視化を行い，学習されたモデルを分析する．

## モデルの読み込みとデータセットの準備

まずは評価データセットに対して特徴ベクトルを計算するために，保存したモデルを再度読み込み，評価データセットを作成する．手順はMLPのときと同様である．

In [ ]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        x = self.gap(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x
    
in_channels = 1
num_classes = 10
model = CNN(in_channels=in_channels, num_classes=num_classes)

save_path = 'output/model.pth'
model.load_state_dict(torch.load(save_path))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [2]:
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

transform = transforms.Compose([
    transforms.ToTensor(),
])

test_dataset = datasets.MNIST(
    root='./data', transform=transform, train=False, download=True)
test_loader = DataLoader(test_dataset, batch_size=100)

## 特徴ベクトルの可視化

MLPと同様に特徴ベクトルを可視化する．モデルの定義からもわかるように，モデルの出力はロジットである．今回は全ての評価データセットに対してロジットを計算し，保存したいので評価と予測で行ったときと同じように，forループを作成する．

ロジットとそのラベルの保存にはリストを利用する．手間なのでここでロジットをCPUに移動し，numpy形式で保存しておく．

In [3]:
logits, labels = [], []

model.eval()
for batch in test_loader:
    x, y = batch
    x, y = x.to(device), y.to(device)
    
    with torch.no_grad():
        output = model(x)
        logits.append(output.cpu().numpy())
        labels.append(y.cpu().numpy())

ロジットのリストをnumpy形式に変換する．

In [ ]:
import numpy as np
logits_ = np.array(logits)
print(logits_.shape)

形状を見てもわかるように，（ループ回数，ミニバッチサイズ，出力次元）というテンソルになっている．これを次のように（ループ回数 * ミニバッチサイズ，出力次元）というように（サンプル数，出力次元）という形状に変換する．

In [ ]:
logits_ = logits_.reshape(-1, logits_.shape[2])
logits_.shape

また正解ラベルについても（ループ回数，ミニバッチサイズ）となっているので（ループ回数*ミニバッチサイズ，）という形状のベクトルへ変換する．

In [ ]:
labels_ = np.array(labels).reshape(-1,)
print(labels_.shape)

ここで特徴ベクトルの可視化をしたいが，10クラス分類のためロジットの出力次元が10であり，MLPのような2次元での可視化ができない．このような場合，[PCA](https://scikit-learn.org/dev/modules/generated/sklearn.decomposition.PCA.html) や [t-SNE](https://scikit-learn.org/1.5/modules/generated/sklearn.manifold.TSNE.html) などの次元削減手法を用いて，人間が解釈可能な2次元もしくは3次元空間に変換することが一般的である．これらのアルゴリズムの説明と実装は行わないが，利用するだけなら `scikit-learn` を用いると非常に簡単に利用できる．

次のセルはPCAによる2次元平面への可視化である．

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

X = logits_
y = labels_

pca = PCA(n_components=2)
X_ = pca.fit_transform(X)
    
plt.figure(figsize=(8, 6))
u, counts = np.unique(y, return_counts=True)
for y_ in u:
    plt.scatter(
        X_[y == y_, 0], X_[y == y_, 1],
        s=10, label=f'Class {y_}', alpha=0.5)
plt.title("PCA Visualization")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()

次のセルはt-SNEによる2次元平面への可視化である．サンプル数と次元数に応じては数分かかる．

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

X = logits_
y = labels_

tsne = TSNE(n_components=2)
X_ = tsne.fit_transform(X)
    
plt.figure(figsize=(8, 6))
u, counts = np.unique(y, return_counts=True)
for y_ in u:
    plt.scatter(
        X_[y == y_, 0], X_[y == y_, 1],
        s=10, label=f'Class {y_}', alpha=0.5)
plt.title("TSNE Visualization")
plt.xlabel("t-SNE1")
plt.ylabel("t-SNE2")
plt.legend()

t-SNEの可視化からもわかるように，クラスごとにクラスタが形成され，分類可能な特徴空間が学習できていることがわかる．今回はロジット空間であったが，MLPやCNNの中間層などでもこのような可視化をすると良い．

しかしながら，これらの可視化を過信するのも良くはない．次元削減ということは，特徴ベクトルが持つ情報が損失しており，またクラスタ間の距離やクラスタの大きさは実際のものとは異なる．これらの手法が示すクラスタや距離はあくまで高次元空間を可視化するための近似的な表現であることに注意されたい．

## フィルタの可視化

最初の畳み込み層のフィルタを可視化する．フィルタ（重み）は次のように取得できる．

In [ ]:
conv1_filters = model.conv1.weight.data.cpu().numpy()
print(conv1_filters.shape)
print(conv1_filters)

最初の畳み込み層（`conv1`）は入力チャネルが1，出力チャネルが16，フィルタサイズが3なので，フィルタの形状が`（16, 1, 3, 3）` となっている．この畳み込み層には16枚のフィルタがあるのでこれを全て可視化する．

In [ ]:
filters = conv1_filters

fig, axes = plt.subplots(4, 4, figsize=(8, 8))
fig.suptitle("Convolution Filters (16 filters of size 3x3)")
for i, ax in enumerate(axes.flat):
    filter_2d = filters[i, 0]
    img = ax.imshow(filter_2d, cmap='gray')
    ax.axis('off') 

## 特徴マップの可視化

入力が画像ということもあり，特徴マップの可視化は重要である．`test_dataset`から取り出した画像を一枚順伝播したとき，上記で可視化したフィルタを適用するとどのような特徴マップが得られるかを可視化しよう．

中間層で計算される特徴マップを取得するもっとも簡単な方法は `output` に加えて，その特徴マップを `return` することであるが，今回はフックというPyTorchの機能を使って取得する．

この機能は次のように `register_forward_hook` を用いて，指定の層の特徴マップを取得する．

In [11]:
feature_maps = []
def hook_function(module, input, output):
    feature_maps.append(output)
    
model.conv1.register_forward_hook(hook_function)

for batch in test_loader:
    x, _ = batch
    x = x.to(device)
    
    with torch.no_grad():
        _ = model(x)
    break

これを実行すると，`feature_maps` 内に特徴マップが保存される．形状を確認すると，（ミニバッチサイズ，出力チャネル，幅，高さ）である．

In [ ]:
feature_maps[0].shape

この特徴マップの最初のサンプルの特徴マップを取得し，フィルタと同様の方法で可視化する．

In [ ]:
feature_map = feature_maps[0][0]

fig, axes = plt.subplots(4, 4, figsize=(8, 8))
fig.suptitle("Feature Maps from Conv1 Layer")
for i, ax in enumerate(axes.flat):
    feature_map_2d = feature_map[i].cpu().detach().numpy()
    ax.imshow(feature_map_2d, cmap='gray')
    ax.axis('off') 

結果からもわかるように，様々な特徴が強調または抑制された特徴マップが可視化できる．一般的なCNNのモデルはチャネル数も膨大で，Poolingによって低解像度化されるため，入力画像のどこを着目しているかをこれらの可視化から解釈することは難しい．しかしながら，バグチェックも含め，このような可視化は重要であるので必要な場面では確認されたい．

## Saliency Map

Saliency Mapはあるクラスに対応するロジットに対して入力の勾配を計算することで，その値を大きく変化させる画素を可視化する手法である．入力に対する勾配を計算する必要があるので，入力 `x` に対して，`x.requires_grad = True` を指定する．そして，予測クラスの特徴次元に対して逆伝播を実行する．得られた勾配の絶対値をmin-max正規化してSaliency Mapとする．

In [14]:
loss_function = nn.CrossEntropyLoss()

for batch in test_loader:
    x, y = batch
    x, y = x.to(device), y.to(device)
    x, y = x[0].unsqueeze(0), y[0]
    
    x.requires_grad = True
    
    model.zero_grad()
    output = model(x)
    prediction = output.argmax(dim=1).item()
    output[0, prediction].backward()
    
    grad = x.grad.data.cpu()
    smap = grad.abs().numpy()
    smap = (smap - smap.min()) / (smap.max() - smap.min())

    break

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(1, 2, 1)
plt.imshow(x.cpu().detach().numpy().reshape(28, 28), cmap='gray')
plt.title(f'Input Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(smap.squeeze(), cmap='hot')
plt.title(f"Saliency Map for Predicted Class {prediction}")
plt.axis("off")
plt.show()

値が大きい画素は7の予測値を大きく変化させる画素であることを示す．結果からもわかるように，7の形がぼんやりと浮かび上がっていることがわかる．さらに良い可視化を得る手法としてGradCAMがあるので興味がある方は調べることをお勧めする．